In [ ]:
# nltk 다운로드 (처음 한 번만 실행)
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
import random
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

# CSV 파일 읽기
df = pd.read_csv('train.csv')

def get_synonym(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return list(set(synonyms))

def augment_sentence(sentence):
    words = word_tokenize(sentence)
    augmented_words = []

    for word in words:
        if random.random() < 0.3:  # 30% 확률로 단어를 대체
            synonyms = get_synonym(word)
            if synonyms:
                augmented_words.append(random.choice(synonyms))
            else:
                augmented_words.append(word)
        else:
            augmented_words.append(word)

    return ' '.join(augmented_words)

# 데이터 증강
augmented_samples = df.copy()
augmented_samples['sentence_1'] = augmented_samples['sentence_1'].apply(augment_sentence)
augmented_samples['sentence_2'] = augmented_samples['sentence_2'].apply(augment_sentence)

# 원본 데이터와 증강 데이터 결합
augmented_df = pd.concat([df, augmented_samples], ignore_index=True)

# 새로운 CSV 파일로 저장
augmented_df.to_csv('SR_train_1.csv', index=False)

In [ ]:
import pandas as pd
import random
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

# CSV 파일 읽기
df = pd.read_csv('dev.csv')

def get_synonym(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return list(set(synonyms))

def augment_sentence(sentence):
    words = word_tokenize(sentence)
    augmented_words = []

    for word in words:
        if random.random() < 0.3:  # 30% 확률로 단어를 대체
            synonyms = get_synonym(word)
            if synonyms:
                augmented_words.append(random.choice(synonyms))
            else:
                augmented_words.append(word)
        else:
            augmented_words.append(word)

    return ' '.join(augmented_words)

# 데이터 증강
augmented_samples = df.copy()
augmented_samples['sentence_1'] = augmented_samples['sentence_1'].apply(augment_sentence)
augmented_samples['sentence_2'] = augmented_samples['sentence_2'].apply(augment_sentence)

# 원본 데이터와 증강 데이터 결합
augmented_df = pd.concat([df, augmented_samples], ignore_index=True)

# 새로운 CSV 파일로 저장
augmented_df.to_csv('SR_dev_1.csv', index=False)

In [ ]:
import os
import requests

def get_papago_translation(text, source_lang='ko', target_lang='ko'):
    url = "https://openapi.naver.com/v1/papago/n2mt"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Naver-Client-Id": "YOUR_CLIENT_ID",
        "X-Naver-Client-Secret": "YOUR_CLIENT_SECRET"
    }
    data = {
        "source": source_lang,
        "target": target_lang,
        "text": text
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json().get('message').get('result').get('translatedText')
    else:
        return None

# 예시 문장
text = "행복하다"
translated_text = get_papago_translation(text)
print(translated_text)

None


In [ ]:
import requests

def kakao_translate(text, src_lang='kr', target_lang='kr'):
    # 카카오 API URL 및 헤더 설정
    url = "https://kapi.kakao.com/v2/translation/translate"
    headers = {
        "Authorization": "KakaoAK YOUR_REST_API_KEY"  # REST API 키 입력
    }
    data = {
        "src_lang": src_lang,
        "target_lang": target_lang,
        "query": text
    }

    # POST 요청을 통해 API 호출
    response = requests.post(url, headers=headers, data=data)

    if response.status_code == 200:
        result = response.json()
        translated_text = result['translated_text'][0][0]
        return translated_text
    else:
        return "API 호출 실패: " + str(response.status_code)

# 예시 문장
text = "나는 자동차를 운전하고 있다"
translated_text = kakao_translate(text)
print(translated_text)

API 호출 실패: 404


In [ ]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 1.1 MB/s eta 0:00:00


In [ ]:
import random
from konlpy.tag import Okt
from nltk.corpus import wordnet

okt = Okt()

# RD: Random Deletion
def random_deletion(words, p=0.1):
    if len(words) == 1:  # 단어가 하나뿐일 때는 삭제하지 않음
        return words
    return [word for word in words if random.uniform(0, 1) > p]

# RS: Random Swap
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# SR: Synonym Replacement
def synonym_replacement(words, n=1):
    new_words = words.copy()
    for _ in range(n):
        word_to_replace = random.choice(new_words)
        synonyms = get_synonyms(word_to_replace)
        if len(synonyms) > 0:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == word_to_replace else word for word in new_words]
    return new_words

def get_synonyms(word):
    # 예시로 고정된 동의어 목록을 사용합니다. (더 나은 방법은 지식 기반을 사용하는 것)
    synonyms_dict = {
        '간단한': ['쉬운', '간편한'],
        '예시': ['사례', '예'],
        '문장': ['문구', '문장'],
        '텍스트': ['글', '문서'],
        '증강': ['증대', '확장']
    }
    return synonyms_dict.get(word, [])

# RI: Random Insertion
def random_insertion(words, n=1):
    new_words = words.copy()
    for _ in range(n):
        new_word = get_random_word(new_words)
        random_idx = random.randint(0, len(new_words))
        new_words.insert(random_idx, new_word)
    return new_words

def get_random_word(words):
    random_word = random.choice(words)
    synonyms = get_synonyms(random_word)
    if len(synonyms) > 0:
        return random.choice(synonyms)
    else:
        return random_word

# 테스트용 예시 문장
sentence = "이것은 텍스트 증강을 위한 간단한 예시 문장입니다".split()

# 기법별 적용 예시
print("Original:", sentence)
print("Random Deletion:", random_deletion(sentence))
print("Random Swap:", random_swap(sentence, n=2))
print("Synonym Replacement:", synonym_replacement(sentence, n=2))
print("Random Insertion:", random_insertion(sentence, n=2))

Original: ['이것은', '텍스트', '증강을', '위한', '간단한', '예시', '문장입니다']
Random Deletion: ['이것은', '텍스트', '증강을', '위한', '예시', '문장입니다']
Random Swap: ['문장입니다', '위한', '증강을', '텍스트', '간단한', '예시', '이것은']
Synonym Replacement: ['이것은', '텍스트', '증강을', '위한', '간편한', '예시', '문장입니다']
Random Insertion: ['이것은', '문장입니다', '텍스트', '증강을', '위한', '간단한', '예시', '이것은', '문장입니다']


In [ ]:
import requests

def get_synonyms_from_naver(word):
    client_id = 'YOUR_CLIENT_ID'
    client_secret = 'YOUR_CLIENT_SECRET'
    url = f'https://openapi.naver.com/v1/search/keyword.json?query={word}'
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret,
    }
    response = requests.get(url, headers=headers)
    return response.json()  # 응답 처리

# 사용 예시
synonyms = get_synonyms_from_naver("간단한")
print(synonyms)

{'errorMessage': 'NID AUTH Result Invalid (1000) : Authentication failed. (인증에 실패했습니다.)', 'errorCode': '024'}


In [ ]:
import nltk
nltk.download('korean_wn')

from nltk.corpus import wordnet as wn

def get_korean_synonyms(word):
    synonyms = []
    for syn in wn.synsets(word, lang='kor'):
        for lemma in syn.lemmas(lang='kor'):
            synonyms.append(lemma.name())
    return list(set(synonyms))  # 중복 제거

# 사용 예시
word = "간단한"
synonyms = get_korean_synonyms(word)
print(synonyms)

[nltk_data] Error loading korean_wn: Package 'korean_wn' not found in
[nltk_data]     index


LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)

	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return

	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def augment_sentence(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	# sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
#	for _ in range(num_new_per_technique):
#		a_words = random_insertion(words, n_ri)
#		augmented_sentences.append(' '.join(a_words))

	# rs
#	for _ in range(num_new_per_technique):
#		a_words = random_swap(words, n_rs)
#		augmented_sentences.append(" ".join(a_words))

	# rd
#	for _ in range(num_new_per_technique):
#		a_words = random_deletion(words, p_rd)
#		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-34-b8efcd2571b1>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [ ]:
df = pd.read_csv('train.csv')

augmented_samples = df.copy()
augmented_samples['sentence_1'] = augmented_samples['sentence_1'].apply(augment_sentence)
augmented_samples['sentence_2'] = augmented_samples['sentence_2'].apply(augment_sentence)

augmented_samples.to_csv('SR_train_2.csv', index=False)